In [52]:
from sklearn.model_selection import GridSearchCV
from pandas import read_csv
from sklearn import svm
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, roc_curve

## Read Data

In [42]:
# Read data
train_data = pd.read_csv("/Users/alex/Desktop/data/5_train_dataset.csv")
X_train = train_data.drop('is_fraud', axis=1)
y_train = train_data['is_fraud']

test_data = pd.read_csv("/Users/alex/Desktop/data/4_test_dataset.csv")
X_test = test_data.drop('is_fraud', axis=1)
y_test = test_data['is_fraud']

## Training LightGBM model

In [43]:
def train_model_lgb(X_train,y_train, lgb_model):

    # model parameters
    param_grid = {
        'num_leaves': [20, 30, 40],
        'learning_rate': [0.01, 0.1],
        'n_estimators': [50, 100, 150],
    }

    # perform grid search for hyperparameter tuning
    grid_search = GridSearchCV(
        lgb_model,
        param_grid,
        cv=3,
        n_jobs=-1
    )

     # fit the grid search on the training data
    grid_search.fit(X_train, y_train)

    # get the best hyperparameters
    best_params = grid_search.best_params_

    # print the best hyperparameters found
    print("Best Hyperparameters:", best_params)

    # set the best hyperparameters to the model
    lgb_model.set_params(**best_params)

    # train the final LightGBM model on the entire training set
    lgb_model.fit(X_train, y_train)




In [56]:
def evaluate_model(model, predX, predY):

    y_pred = model.predict(predX)

    # accuracy
    acc =  accuracy_score(predY, y_pred)
    print("Accuracy: %.2f%%" % (acc * 100))

    # precision
    precision = precision_score(predY, y_pred)
    print("Precision: %.2f" % precision)

    # recall
    recall = recall_score(predY, y_pred)
    print("Recall: %.2f" % recall)

    # f1 Score
    f1 = f1_score(predY, y_pred)
    print("F1 Score: %.2f" % f1)

    # ROC-AUC score
    roc_auc = roc_auc_score(predY, y_pred)
    print("ROC-AUC Score: %.2f" % roc_auc)


In [50]:

# create the LightGBM model
lgb_model = lgb.LGBMClassifier()

# train model
train_model_lgb(X_train,y_train, lgb_model)


[LightGBM] [Info] Number of positive: 871, number of negative: 3055
[LightGBM] [Info] Number of positive: 872, number of negative: 3054
[LightGBM] [Info] Number of positive: 871, number of negative: 3055
[LightGBM] [Info] Number of positive: 871, number of negative: 3055
[LightGBM] [Info] Number of positive: 872, number of negative: 3054
[LightGBM] [Info] Number of positive: 871, number of negative: 3055
[LightGBM] [Info] Number of positive: 871, number of negative: 3055
[LightGBM] [Info] Number of positive: 872, number of negative: 3054
[LightGBM] [Info] Number of positive: 872, number of negative: 3054
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3861
[LightGBM] [Info] Number of data points in the train set: 3926, number of used features: 20
[LightGBM] [Info] Auto-choosing row-

In [57]:
evaluate_model(lgb_model,X_test, y_test)

Accuracy: 97.53%
Precision: 0.98
Recall: 0.91
F1 Score: 0.94
ROC-AUC Score: 0.95
